<a href="https://colab.research.google.com/github/Rey1380/Speech-Recognition-application-in-a-computer-game/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import all needed libraries


In [ ]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
from pydub import AudioSegment
from python_speech_features import mfcc
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tkinter import *
from tkinter import ttk
import torch
import librosa
import IPython.display as ipd
import numpy as np
import os
import gradio as gr
import speech_recognition as sr
import warnings
import soundfile as sf
import sounddevice as sd
import pygame, os, random
import tkinter as tk
import pygame, os, random
import tkinter as tk




# Preprocess input data(down)



In [ ]:
input_dir = r'E:\classes\multimedia\final project\voices\down'
output_dir = "E:\\classes\\multimedia\\final project\\voices\\output16000"
target_duration = 4000  # target duration in milliseconds

for fname in next(os.walk(input_dir))[2]:
    spk_id, _ = fname.split("_")
    speech_array, sample_rate = librosa.load((os.path.join(input_dir, fname)),sr=16000) #load data
    speech_array = speech_array.squeeze().astype(np.float64)

# Get a list of all audio files in the input directory
audio_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

# Process each audio file and save it to the output directory
for file in audio_files:
    audio = AudioSegment.from_wav(os.path.join(input_dir, file))
    length = len(audio)

    if length < target_duration:
        # Zero-pad the audio file
        padding = AudioSegment.silent(duration=target_duration - length)
        audio = audio + padding
    elif length > target_duration:
        # Trim the audio file
        audio = audio[:target_duration]

    # Save the output file
    output_file = os.path.join(output_dir, file)
    audio.export(output_file, format="wav")

# Preprocess input data(up)



In [ ]:
input_dir = r'E:\classes\multimedia\final project\voices\up'
output_dir = "E:\\classes\\multimedia\\final project\\voices\\output16000"
target_duration = 4000  # target duration in milliseconds

for fname in next(os.walk(input_dir))[2]:
    spk_id, _ = fname.split("_")
    speech_array, sample_rate = librosa.load((os.path.join(input_dir, fname)),sr=16000) #load data
    speech_array = speech_array.squeeze().astype(np.float64)

# Get a list of all audio files in the input directory
audio_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

# Process each audio file and save it to the output directory
for file in audio_files:
    audio = AudioSegment.from_wav(os.path.join(input_dir, file))
    length = len(audio)

    if length < target_duration:
        # Zero-pad the audio file
        padding = AudioSegment.silent(duration=target_duration - length)
        audio = audio + padding
    elif length > target_duration:
        # Trim the audio file
        audio = audio[:target_duration]

    # Save the output file
    output_file = os.path.join(output_dir, file)
    audio.export(output_file, format="wav")

# Preprocess input data(left)



In [ ]:
input_dir = r'E:\classes\multimedia\final project\voices\left'
output_dir = "E:\\classes\\multimedia\\final project\\voices\\output16000"
target_duration = 4000  # target duration in milliseconds

for fname in next(os.walk(input_dir))[2]:
    spk_id, _ = fname.split("_")
    speech_array, sample_rate = librosa.load((os.path.join(input_dir, fname)),sr=16000) #load data
    speech_array = speech_array.squeeze().astype(np.float64)

# Get a list of all audio files in the input directory
audio_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

# Process each audio file and save it to the output directory
for file in audio_files:
    audio = AudioSegment.from_wav(os.path.join(input_dir, file))
    length = len(audio)

    if length < target_duration:
        # Zero-pad the audio file
        padding = AudioSegment.silent(duration=target_duration - length)
        audio = audio + padding
    elif length > target_duration:
        # Trim the audio file
        audio = audio[:target_duration]

    # Save the output file
    output_file = os.path.join(output_dir, file)
    audio.export(output_file, format="wav")

# Preprocess input data(right)



In [ ]:
input_dir = r'E:\classes\multimedia\final project\voices\right'
output_dir = "E:\\classes\\multimedia\\final project\\voices\\output16000"
target_duration = 4000  # target duration in milliseconds

for fname in next(os.walk(input_dir))[2]:
    spk_id, _ = fname.split("_")
    speech_array, sample_rate = librosa.load((os.path.join(input_dir, fname)),sr=16000) #load data
    speech_array = speech_array.squeeze().astype(np.float64)

# Get a list of all audio files in the input directory
audio_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

# Process each audio file and save it to the output directory
for file in audio_files:
    audio = AudioSegment.from_wav(os.path.join(input_dir, file))
    length = len(audio)

    if length < target_duration:
        # Zero-pad the audio file
        padding = AudioSegment.silent(duration=target_duration - length)
        audio = audio + padding
    elif length > target_duration:
        # Trim the audio file
        audio = audio[:target_duration]

    # Save the output file
    output_file = os.path.join(output_dir, file)
    audio.export(output_file, format="wav")

#  MLP network and MFCC coefficients for voice command recognition(not real time)

In [ ]:
class VoiceCommandSystem:
    def __init__(self, data_path, model_path):
        self.data_path = data_path
        self.model_path = model_path
        self.mlp_model = None
        self.scaler = StandardScaler()
        self.player_position = [0, 0]

    def move_up(self):
        self.player_position[1] += 1
        print("Moving the player up. Player position:", self.player_position)

    def move_down(self):
        self.player_position[1] -= 1
        print("Moving the player down. Player position:", self.player_position)

    def move_left(self):
        self.player_position[0] -= 1
        print("Moving the player left. Player position:", self.player_position)

    def move_right(self):
        self.player_position[0] += 1
        print("Moving the player right. Player position:", self.player_position)

    def extract_features(self, audio):
        mfcc = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=26)
        return mfcc.flatten()

    def load_data(self):
        voice_files = os.listdir(self.data_path)
        X = []
        y = []

        for file in voice_files:
            file_path = os.path.join(self.data_path, file)
            audio, _ = librosa.load(file_path, sr=16000)
            feature = self.extract_features(audio)
            X.append(feature)
            y.append(file.split("_")[0])

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_val_scaled = self.scaler.transform(X_val)
        return X_train_scaled, X_val_scaled, y_train, y_val

    def train_model(self):
        X_train, X_val, y_train, y_val = self.load_data()
        best_val_acc = 0
        self.mlp_model = MLPClassifier(hidden_layer_sizes=(256, 128), alpha=0.0001, random_state=42)
        self.mlp_model.fit(X_train, y_train)

        # Calculate accuracy on training set
        train_pred = self.mlp_model.predict(X_train)
        train_acc = accuracy_score(y_train, train_pred)
        print(f"Training Accuracy: {train_acc}")

        # Calculate accuracy on validation set
        val_pred = self.mlp_model.predict(X_val)
        val_acc = accuracy_score(y_val, val_pred)
        print(f" Validation Accuracy: {val_acc}")

        if val_acc > best_val_acc:
             best_val_acc = val_acc
             # Save the best model if desired

        return best_val_acc

    def recognize_command(self, file_path):
        audio, _ = librosa.load(file_path, sr=16000)
        feature = self.extract_features(audio)
        feature_scaled = self.scaler.transform([feature])
        predicted_label = self.mlp_model.predict(feature_scaled)[0]

        if predicted_label == "up":
            self.move_up()
        elif predicted_label == "down":
            self.move_down()
        elif predicted_label == "left":
            self.move_left()
        elif predicted_label == "right":
            self.move_right()

        player_position_str = f"Player position: {self.player_position}"
        return predicted_label,player_position_str

if __name__ == '__main__':
    data_path = "E:\\classes\\multimedia\\final project\\voices\\output16000"
    model_path =  "E:\\classes\\multimedia\\final project\\model"
    player_position = [0, 0]

    voice_command_system = VoiceCommandSystem(data_path, model_path)
    best_val_acc = voice_command_system.train_model()

    audio_input = gr.inputs.Audio(label="Speak Command", type="filepath")
    label_output = gr.outputs.Textbox(label="Recognized Command")

    iface = gr.Interface(fn=voice_command_system.recognize_command, inputs=audio_input, outputs=label_output)
    iface.launch()

Training Accuracy: 1.0
 Validation Accuracy: 0.9444444444444444


C:\Users\Lenovo\AppData\Roaming\Python\Python39\site-packages\gradio\inputs.py:321: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\Lenovo\AppData\Roaming\Python\Python39\site-packages\gradio\inputs.py:324: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(source=source, type=type, label=label, optional=optional)
C:\Users\Lenovo\AppData\Roaming\Python\Python39\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python39\site-packages\gradio\routes.py", line 437, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python39\site-packages\gradio\blocks.py", line 1352, in process_api
    result = await self.call_function(
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python39\site-packages\gradio\blocks.py", line 1077, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "D:\anaconda\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "D:\anaconda\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "D:\anaconda\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel

Moving the player right. Player position: [1, 0]


# Using MLP network and MFCC coefficients for voice command recognition(real time)

In [ ]:
#real time code
class VoiceCommandSystem:
    def __init__(self, data_path, model_path):
        self.data_path = data_path
        self.model_path = model_path
        self.mlp_model = None
        self.scaler = StandardScaler()
        self.player_position = [0, 0]

    def move_up(self):
        self.player_position[1] += 1
        print("Moving the player up. Player position:", self.player_position)

    def move_down(self):
        self.player_position[1] -= 1
        print("Moving the player down. Player position:", self.player_position)

    def move_left(self):
        self.player_position[0] -= 1
        print("Moving the player left. Player position:", self.player_position)

    def move_right(self):
        self.player_position[0] += 1
        print("Moving the player right. Player position:", self.player_position)

    def extract_features(self, audio):
        mfcc = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=26)
        return mfcc.flatten()

    def load_data(self):
        voice_files = os.listdir(self.data_path)
        X = []
        y = []

        for file in voice_files:
            file_path = os.path.join(self.data_path, file)
            audio, _ = librosa.load(file_path, sr=16000)
            feature = self.extract_features(audio)
            X.append(feature)
            y.append(file.split("_")[0])

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_val_scaled = self.scaler.transform(X_val)
        return X_train_scaled, X_val_scaled, y_train, y_val

    def train_model(self):
        X_train, X_val, y_train, y_val = self.load_data()
        best_val_acc = 0
        self.mlp_model = MLPClassifier(hidden_layer_sizes=(256, 128), alpha=0.0001, random_state=42)
        self.mlp_model.fit(X_train, y_train)

        # Calculate accuracy on training set
        train_pred = self.mlp_model.predict(X_train)
        train_acc = accuracy_score(y_train, train_pred)
        print(f"Training Accuracy: {train_acc}")

        # Calculate accuracy on validation set
        val_pred = self.mlp_model.predict(X_val)
        val_acc = accuracy_score(y_val, val_pred)
        print(f"Validation Accuracy: {val_acc}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            # Save the best model if desired

        return best_val_acc

    def recognize_command(self, audio_path):
        audio, _ = librosa.load(audio_path, sr=16000)
        feature = self.extract_features(audio)
        feature_scaled = self.scaler.transform([feature])
        predicted_label = self.mlp_model.predict(feature_scaled)[0]

        if predicted_label == "up":
            self.move_up()
        elif predicted_label == "down":
            self.move_down()
        elif predicted_label == "left":
            self.move_left()
        elif predicted_label == "right":
            self.move_right()

        player_position_str = f"Player position: {self.player_position}"
        return predicted_label, player_position_str


if __name__ == '__main__':
    data_path = "E:\\classes\\multimedia\\final project\\voices\\output16000"
    model_path = "E:\\classes\\multimedia\\final project\\model"
    player_position = [0, 0]
    voice_command_system = VoiceCommandSystem(data_path, model_path)
    best_val_acc = voice_command_system.train_model()
    samplerate = 16000  # Hertz
    duration = 3  # seconds
    filename = r'E:\classes\multimedia\final project\recorded\down_200.wav'
    mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,channels=2, blocking=True)
    sf.write(filename, mydata, samplerate)
    input_dir = r'E:\classes\multimedia\final project\recorded'
    output_dir= r'E:\classes\multimedia\final project'
    target_duration = 4000  # target duration in milliseconds

    for fname in next(os.walk(input_dir))[2]:
        spk_id, _ = fname.split("_")
        speech_array, sample_rate = librosa.load((os.path.join(input_dir, fname)),sr=16000) #load data
        speech_array = speech_array.squeeze().astype(np.float64)

    # Get a list of all audio files in the input directory
    audio_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

    # Process each audio file and save it to the output directory
    for file in audio_files:
        audio = AudioSegment.from_wav(os.path.join(input_dir, file))
        length = len(audio)

        if length < target_duration:
            # Zero-pad the audio file
            padding = AudioSegment.silent(duration=target_duration - length)
            audio = audio + padding
        elif length > target_duration:
            # Trim the audio file
            audio = audio[:target_duration]

        # Save the output file
        output_file = os.path.join(output_dir, file)
        audio.export(output_file, format="wav")




    a,b=voice_command_system.recognize_command(r'E:\classes\multimedia\final project\down_200.wav')
    print(a,b)

Training Accuracy: 1.0
Validation Accuracy: 0.9444444444444444
Moving the player left. Player position: [-1, 0]
left Player position: [-1, 0]


# Game that works with keyboard keys

In [ ]:
class Player:
    vel_x = 0
    vel_y = 0
    acc = 0.2

    def __init__(self, x, y, width, height):
        self.shots = 0
        self.shoots = []
        self.x, self.y, self.width, self.height = x, y, width, height
        self.image = pygame.transform.scale(
            pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\player.png"), (width, height)
        )

    def move(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_z]:
            shoot = Shoot(self)
            self.shoots.append(shoot)
        if self.y > 0:
            if keys[pygame.K_UP]:
                self.vel_y -= self.acc
        else:
            if not keys[pygame.K_DOWN]:
                self.y = 0
                self.vel_y = 0
        if self.y + self.height < HEIGHT:
            if keys[pygame.K_DOWN]:
                self.vel_y += self.acc
        else:
            if not keys[pygame.K_UP]:
                self.y = HEIGHT - self.height
                self.vel_y = 0
        if PLAYER_BACKFORTH:
            if self.x > 0:
                if keys[pygame.K_LEFT]:
                    self.vel_x -= self.acc
            else:
                if not keys[pygame.K_RIGHT]:
                    self.x = 0
                    self.vel_x = 0
            if self.x + self.width < WIDTH:
                if keys[pygame.K_RIGHT]:
                    self.vel_x += self.acc
            else:
                if not keys[pygame.K_LEFT]:
                    self.x = WIDTH - self.width
                    self.vel_x = 0

        self.vel_x = min(5, self.vel_x)
        self.vel_y = min(5, self.vel_y)
        self.vel_x = max(-5, self.vel_x)
        self.vel_y = max(-5, self.vel_y)

        self.x += self.vel_x
        self.y += self.vel_y

    def draw(self, win):
        win.blit(self.image, (round(self.x), round(self.y)))

    def get_mask(self):
        return pygame.mask.from_surface(self.image)


class Heart:
    def __init__(self):
        self.heart = 3
        self.x = 25
        self.y = 25
        self.image = pygame.transform.scale(
            pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\heart.png"), (25, 25)
        )

    def draw(self, win):
        for heart_num in range(self.heart):
            win.blit(self.image, (round(self.x) + heart_num * 30, round(self.y)))


class Shoot:
    WIDTH, HEIGHT = 5, 5
    VEL = 20

    def __init__(self, player):
        self.x, self.y = player.x + 80, player.y + 30
        self.image = pygame.transform.scale(
            pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\shoot.png"),
            (self.WIDTH, self.HEIGHT),
        )

    def move(self, player, bullets):
        self.x += self.VEL

        for bullet in bullets:
            if self.collide(bullet):
                bullets.remove(bullet)
                player.shoots.remove(self)
                return "crash"

    def draw(self, win):
        win.blit(self.image, (self.x, self.y))

    def collide(self, bullet):
        bullet_mask = bullet.get_mask()
        mask = pygame.mask.from_surface(self.image)
        top_offset = (round(self.x - bullet.x), round(self.y - bullet.y))
        top_point = bullet_mask.overlap(mask, top_offset)

        if top_point:
            return True

        return False


class Bullet:
    WIDTH, HEIGHT = 80, 30
    VEL = 5

    def __init__(self, y):
        self.x, self.y = WIDTH + random.randint(0, 800), y
        self.image = pygame.transform.scale(
            pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\bullet.png"),
            (self.WIDTH, self.HEIGHT),
        )

    def move(self, player):
        self.x -= self.VEL

        if self.collide(player):
            return "crash"
        if self.x + self.WIDTH < 0:
            return "out"

        return False

    def draw(self, win):
        win.blit(self.image, (self.x, self.y))

    def collide(self, player):
        player_mask = player.get_mask()
        mask = pygame.mask.from_surface(self.image)
        top_offset = (round(self.x - player.x), round(self.y - player.y))
        top_point = player_mask.overlap(mask, top_offset)

        if top_point:
            return True

        return False

    def get_mask(self):
        return pygame.mask.from_surface(self.image)


class Clouds:
    VEL = 2

    def __init__(self):
        self.y = 0
        self.x1 = 0
        self.image = pygame.transform.scale(
            pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\clouds_bg.png"), (WIDTH, HEIGHT)
        )
        self.width = self.image.get_width()
        self.x2 = self.width

    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL

        if self.x1 + self.width < 0:
            self.x1 = self.x2 + self.width

        if self.x2 + self.width < 0:
            self.x2 = self.x1 + self.width

    def draw(self, win):
        win.blit(self.image, (self.x1, self.y))
        win.blit(self.image, (self.x2, self.y))


def draw_window(win, hearts, player, shoots, clouds, bullets, score, max_score, playing):
    clouds.draw(win)
    player.draw(win)
    hearts.draw(win)
    for shoot in shoots:
        shoot.draw(win)
    for bullet in bullets:
        bullet.draw(win)
    text1 = FONT.render(f"Max Score: {max_score}", 1, BLACK)
    text2 = FONT.render(f"Score: {score}", 1, BLACK)
    text3 = FONT.render(f"Shots: {player.shots}", 1, BLACK)
    win.blit(text1, (WIDTH - text1.get_width() - 1, 5))
    win.blit(text2, (WIDTH - text2.get_width() - 5, 10 + text1.get_height()))
    win.blit(text3, (WIDTH - text3.get_width() - 5, 60 + text2.get_height()))
    if not playing:
        text = BIG_FONT.render("Press SPACEBAR to play", 1, BLACK)
        win.blit(
            text,
            (WIDTH // 2 - text.get_width() // 2, HEIGHT // 2 - text.get_height() // 2),
        )


def create_bullets(n):
    bullets = []
    for _ in range(round(n)):
        bullets.append(Bullet(random.randint(0, HEIGHT - 15)))

    return bullets


def adjust_bullets(bullets, n):
    if len(bullets) > round(n):
        bullets.pop(0)
    elif len(bullets) < round(n):
        bullets.append(Bullet(random.randint(0, HEIGHT - 15)))

    return bullets


def end_menu(score):
    global WINDOW, username

    #Create an instance of Tkinter frame
    win = Tk()
    win.title("Game Over!")

    #Set the geometry of Tkinter frame
    win.geometry("550x150")

    def start_clicked():
        global FONT, BIG_FONT
        win.destroy()

        pygame.init()

        os.environ["SDL_VIDEO_WINDOW_POS"] = "%d,%d" % (
        screen_width // 2 - WIDTH // 2,
        screen_height // 2 - HEIGHT // 2,
        )

        WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Dodger Plane")
        pygame.display.set_icon(pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\icon.png"))
        FONT = pygame.font.SysFont("comicsans", 30)
        BIG_FONT = pygame.font.SysFont("comicsans", 70)

        main(WINDOW, username)
        return

    # def score_clicked():
    #     return

    #Initialize a Label to display the User Input
    label=Label(win, text="Score : " + str(score), font=("Courier 18 bold"))
    label.pack(pady=20)

    # Add an empty label to create vertical spacing
    empty_label = Label(win, height=1)
    empty_label.pack()

    #Create a Button to validate Entry Widget
    button_start = ttk.Button(win, text="Start Again", width= 20, command=start_clicked)
    button_start.place(relx=0.5, rely=0.55, anchor="center")

    # button_score = ttk.Button(win, text="Score Board", width= 20, command=score_clicked)
    # button_score.place(relx=0.5, rely=0.65, anchor="center")

    win.mainloop()

    return

def move_bullets(hearts, player, shoots, bullets, score, n):
    rem = []
    for bullet in bullets:
        movement = bullet.move(player)
        if movement == "out":
            rem.append(bullet)
            score += 1
        if movement == "crash":
            hearts.heart -= 1
            bullets.remove(bullet)
            if hearts.heart == 0:
                pygame.quit()
                # win.quit()
                with open("scores.txt", "a") as f:
                    f.write(username + ": score = " + str(score) + ", successful shots = " + str(player.shots) + "\n")
                end_menu(score)
                return
    for shoot in shoots:
        movement = shoot.move(player, bullets)
        if movement == "crash":
            score += 10
            player.shots += 1
    for bullet in rem:
        try:
            bullets.remove(bullet)
        except ValueError:
            pass

    while len(bullets) != round(n):
        bullets = adjust_bullets(bullets, n)

    return hearts, player, shoots, bullets, score, n


def move_objs(hearts, player, shoots, bullets, score, n, clouds):
    player.move()
    clouds.move()
    return move_bullets(hearts, player, shoots, bullets, score, n)


def main(win, username):
    clock = pygame.time.Clock()
    hearts = Heart()
    player = Player(random.randint(5, 35), random.randint(0, HEIGHT - 50), 100, 50)
    clouds = Clouds()
    shoots = player.shoots
    n = 5
    bullets = create_bullets(n)
    score = 0
    try:
        with open("max_score.txt", "r") as f:
            line = f.readline()
            max_score = int(line)
    except FileNotFoundError:
        max_score = 0
    run = True
    playing = False
    while run:
        clock.tick(FPS)
        draw_window(win, hearts, player, shoots, clouds, bullets, score, max_score, playing)
        n = min(30, n)
        if playing:
            n += 0.001
            try:
                hearts, player, shoots, bullets, score, n = move_objs(hearts, player, shoots, bullets, score, n, clouds)
            except:
                break
        if max_score < score:
            max_score = score
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                with open("max_score.txt", "w") as f:
                    f.write(str(max_score))
                run = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    playing = True if playing == False else False
        pygame.display.update()
    return


#Create an instance of Tkinter frame
win = Tk()
win.title("Welcome! ^^")

#Set the geometry of Tkinter frame
win.geometry("550x200")

clicked = False
username = ""

def close():
   global clicked, username
   clicked = True
   username = entry.get()
   win.destroy()

#Initialize a Label to display the User Input
label=Label(win, text="What's your name? :)", font=("Courier 18 bold"))
label.pack(pady=20)

# Add an empty label to create vertical spacing
empty_label = Label(win, height=1)
empty_label.pack()

#Create an Entry widget to accept User Input
entry = Entry(win, width=40)
entry.focus_set()
entry.pack()

#Create a Button to validate Entry Widget
ttk.Button(win, text="Okay", width= 20, command=close).pack(pady=30)

win.mainloop()

WIDTH, HEIGHT = 1000, 700
FPS = 60

if clicked:
    pygame.init()

    screen_width = tk.Tk().winfo_screenwidth()
    screen_height = tk.Tk().winfo_screenheight()

    os.environ["SDL_VIDEO_WINDOW_POS"] = "%d,%d" % (
        screen_width // 2 - WIDTH // 2,
        screen_height // 2 - HEIGHT // 2,
    )
    WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Dodger Plane")
    pygame.display.set_icon(pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\icon.png"))
    PLAYER_BACKFORTH = True
    FONT = pygame.font.SysFont("comicsans", 30)
    BIG_FONT = pygame.font.SysFont("comicsans", 70)


    BLACK = (0, 0, 0)
    main(WINDOW, username)


#  Game that works with Voice commands

In [ ]:
class VoiceCommandSystem:
    def __init__(self, data_path, model_path):
        self.data_path = data_path
        self.model_path = model_path
        self.mlp_model = None
        self.scaler = StandardScaler()
        self.player_position = [0, 0]

    def move_up(self):
        self.player_position[1] += 1
        print("Moving the player up. Player position:", self.player_position)

    def move_down(self):
        self.player_position[1] -= 1
        print("Moving the player down. Player position:", self.player_position)

    def move_left(self):
        self.player_position[0] -= 1
        print("Moving the player left. Player position:", self.player_position)

    def move_right(self):
        self.player_position[0] += 1
        print("Moving the player right. Player position:", self.player_position)

    def extract_features(self, audio):
        mfcc = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=26)
        return mfcc.flatten()

    def load_data(self):
        voice_files = os.listdir(self.data_path)
        X = []
        y = []

        for file in voice_files:
            file_path = os.path.join(self.data_path, file)
            audio, _ = librosa.load(file_path, sr=16000)
            feature = self.extract_features(audio)
            X.append(feature)
            y.append(file.split("_")[0])

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_val_scaled = self.scaler.transform(X_val)
        return X_train_scaled, X_val_scaled, y_train, y_val

    def train_model(self):
        X_train, X_val, y_train, y_val = self.load_data()
        best_val_acc = 0
        self.mlp_model = MLPClassifier(hidden_layer_sizes=(256, 128), alpha=0.0001, random_state=42)
        self.mlp_model.fit(X_train, y_train)

        # Calculate accuracy on training set
        train_pred = self.mlp_model.predict(X_train)
        train_acc = accuracy_score(y_train, train_pred)
        print(f"Training Accuracy: {train_acc}")

        # Calculate accuracy on validation set
        val_pred = self.mlp_model.predict(X_val)
        val_acc = accuracy_score(y_val, val_pred)
        print(f"Validation Accuracy: {val_acc}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            # Save the best model if desired

        return best_val_acc

    def recognize_command(self, audio_path):
        audio, _ = librosa.load(audio_path, sr=16000)
        feature = self.extract_features(audio)
        feature_scaled = self.scaler.transform([feature])
        predicted_label = self.mlp_model.predict(feature_scaled)[0]

        if predicted_label == "up":
            self.move_up()
        elif predicted_label == "down":
            self.move_down()
        elif predicted_label == "left":
            self.move_left()
        elif predicted_label == "right":
            self.move_right()

        player_position_str = f"Player position: {self.player_position}"
        return predicted_label, player_position_str


class Player:
    vel_x = 0
    vel_y = 0
    acc = 0.2

    def __init__(self, x, y, width, height):
        self.shots = 0
        self.shoots = []
        self.x, self.y, self.width, self.height = x, y, width, height
        self.image = pygame.transform.scale(
           pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\player.png"), (width, height)
        )

    def move(self):
        # keys = pygame.key.get_pressed()
        data_path = "E:\\classes\\multimedia\\final project\\voices\\output16000"
        model_path = "E:\\classes\\multimedia\\final project\\model"
        player_position = [0, 0]
        voice_command_system = VoiceCommandSystem(data_path, model_path)
        best_val_acc = voice_command_system.train_model()
        samplerate = 16000  # Hertz
        duration = 3  # seconds

        filename = r'E:\classes\multimedia\final project\recorded\down_200.wav'
        mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,channels=2, blocking=True)
        sf.write(filename, mydata, samplerate)
        input_dir = r'E:\classes\multimedia\final project\recorded'
        output_dir= r'E:\classes\multimedia\final project'
        target_duration = 4000  # target duration in milliseconds

        for fname in next(os.walk(input_dir))[2]:
            spk_id, _ = fname.split("_")
            speech_array, sample_rate = librosa.load((os.path.join(input_dir, fname)),sr=16000) #load data
            speech_array = speech_array.squeeze().astype(np.float64)

        # Get a list of all audio files in the input directory
        audio_files = [f for f in os.listdir(input_dir) if f.endswith(".wav")]

        # Process each audio file and save it to the output directory
        for file in audio_files:
            audio = AudioSegment.from_wav(os.path.join(input_dir, file))
            length = len(audio)

            if length < target_duration:
                # Zero-pad the audio file
                padding = AudioSegment.silent(duration=target_duration - length)
                audio = audio + padding
            elif length > target_duration:
                # Trim the audio file
                audio = audio[:target_duration]

            # Save the output file
            output_file = os.path.join(output_dir, file)
            audio.export(output_file, format="wav")




        a,b=voice_command_system.recognize_command(r'E:\classes\multimedia\final project\down_200.wav')
        # print(a,b)
        if a:
            shoot = Shoot(self)
            self.shoots.append(shoot)
        if self.y > 0:
            if a == "up":
                self.vel_y -= self.acc
        else:
            if not a == "down":
                self.y = 0
                self.vel_y = 0
        if self.y + self.height < HEIGHT:
            if a == "down":
                self.vel_y += self.acc
        else:
            if not a == "up":
                self.y = HEIGHT - self.height
                self.vel_y = 0
        if PLAYER_BACKFORTH:
            if self.x > 0:
                if a == "left":
                    self.vel_x -= self.acc
            else:
                if not a == "right":
                    self.x = 0
                    self.vel_x = 0
            if self.x + self.width < WIDTH:
                if a == "right":
                    self.vel_x += self.acc
            else:
                if not a == "left":
                    self.x = WIDTH - self.width
                    self.vel_x = 0

        self.vel_x = min(5, self.vel_x)
        self.vel_y = min(5, self.vel_y)
        self.vel_x = max(-5, self.vel_x)
        self.vel_y = max(-5, self.vel_y)

        self.x += self.vel_x
        self.y += self.vel_y

    def draw(self, win):
        win.blit(self.image, (round(self.x), round(self.y)))

    def get_mask(self):
        return pygame.mask.from_surface(self.image)


class Heart:
    def __init__(self):
        self.heart = 3
        self.x = 25
        self.y = 25
        self.image = pygame.transform.scale(
            pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\heart.png"), (25, 25)
        )

    def draw(self, win):
        for heart_num in range(self.heart):
            win.blit(self.image, (round(self.x) + heart_num * 30, round(self.y)))


class Shoot:
    WIDTH, HEIGHT = 5, 5
    VEL = 20

    def __init__(self, player):
        self.x, self.y = player.x + 80, player.y + 30
        self.image = pygame.transform.scale(
            pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\shoot.png"),
            (self.WIDTH, self.HEIGHT),
        )

    def move(self, player, bullets):
        self.x += self.VEL

        for bullet in bullets:
            if self.collide(bullet):
                bullets.remove(bullet)
                player.shoots.remove(self)
                return "crash"

    def draw(self, win):
        win.blit(self.image, (self.x, self.y))

    def collide(self, bullet):
        bullet_mask = bullet.get_mask()
        mask = pygame.mask.from_surface(self.image)
        top_offset = (round(self.x - bullet.x), round(self.y - bullet.y))
        top_point = bullet_mask.overlap(mask, top_offset)

        if top_point:
            return True

        return False


class Bullet:
    WIDTH, HEIGHT = 80, 30
    VEL = 5

    def __init__(self, y):
        self.x, self.y = WIDTH + random.randint(0, 800), y
        self.image = pygame.transform.scale(
            pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\bullet.png"),
            (self.WIDTH, self.HEIGHT),
        )

    def move(self, player):
        self.x -= self.VEL

        if self.collide(player):
            return "crash"
        if self.x + self.WIDTH < 0:
            return "out"

        return False

    def draw(self, win):
        win.blit(self.image, (self.x, self.y))

    def collide(self, player):
        player_mask = player.get_mask()
        mask = pygame.mask.from_surface(self.image)
        top_offset = (round(self.x - player.x), round(self.y - player.y))
        top_point = player_mask.overlap(mask, top_offset)

        if top_point:
            return True

        return False

    def get_mask(self):
        return pygame.mask.from_surface(self.image)


class Clouds:
    VEL = 2

    def __init__(self):
        self.y = 0
        self.x1 = 0
        self.image = pygame.transform.scale(
            pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\clouds_bg.png"), (WIDTH, HEIGHT)
        )
        self.width = self.image.get_width()
        self.x2 = self.width

    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL

        if self.x1 + self.width < 0:
            self.x1 = self.x2 + self.width

        if self.x2 + self.width < 0:
            self.x2 = self.x1 + self.width

    def draw(self, win):
        win.blit(self.image, (self.x1, self.y))
        win.blit(self.image, (self.x2, self.y))


def draw_window(win, hearts, player, shoots, clouds, bullets, score, max_score, playing):
    clouds.draw(win)
    player.draw(win)
    hearts.draw(win)
    for shoot in shoots:
        shoot.draw(win)
    for bullet in bullets:
        bullet.draw(win)
    text1 = FONT.render(f"Max Score: {max_score}", 1, BLACK)
    text2 = FONT.render(f"Score: {score}", 1, BLACK)
    text3 = FONT.render(f"Shots: {player.shots}", 1, BLACK)
    win.blit(text1, (WIDTH - text1.get_width() - 1, 5))
    win.blit(text2, (WIDTH - text2.get_width() - 5, 10 + text1.get_height()))
    win.blit(text3, (WIDTH - text3.get_width() - 5, 60 + text2.get_height()))
    if not playing:
        text = BIG_FONT.render("Press SPACEBAR to play", 1, BLACK)
        win.blit(
            text,
            (WIDTH // 2 - text.get_width() // 2, HEIGHT // 2 - text.get_height() // 2),
        )


def create_bullets(n):
    bullets = []
    for _ in range(round(n)):
        bullets.append(Bullet(random.randint(0, HEIGHT - 15)))

    return bullets


def adjust_bullets(bullets, n):
    if len(bullets) > round(n):
        bullets.pop(0)
    elif len(bullets) < round(n):
        bullets.append(Bullet(random.randint(0, HEIGHT - 15)))

    return bullets


def end_menu(score):
    global WINDOW, username

    #Create an instance of Tkinter frame
    win = Tk()
    win.title("Game Over!")

    #Set the geometry of Tkinter frame
    win.geometry("550x150")

    def start_clicked():
        global FONT, BIG_FONT
        win.destroy()

        pygame.init()

        os.environ["SDL_VIDEO_WINDOW_POS"] = "%d,%d" % (
        screen_width // 2 - WIDTH // 2,
        screen_height // 2 - HEIGHT // 2,
        )

        WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Dodger Plane")
        pygame.display.set_icon(r"E:\classes\multimedia\final project\game part\assets\icon.png")
        FONT = pygame.font.SysFont("comicsans", 30)
        BIG_FONT = pygame.font.SysFont("comicsans", 70)

        main(WINDOW, username)
        return

    # def score_clicked():
    #     return

    #Initialize a Label to display the User Input
    label=Label(win, text="Score : " + str(score), font=("Courier 18 bold"))
    label.pack(pady=20)

    # Add an empty label to create vertical spacing
    empty_label = Label(win, height=1)
    empty_label.pack()

    #Create a Button to validate Entry Widget
    button_start = ttk.Button(win, text="Start Again", width= 20, command=start_clicked)
    button_start.place(relx=0.5, rely=0.55, anchor="center")

    # button_score = ttk.Button(win, text="Score Board", width= 20, command=score_clicked)
    # button_score.place(relx=0.5, rely=0.65, anchor="center")

    win.mainloop()

    return

def move_bullets(hearts, player, shoots, bullets, score, n):
    rem = []
    for bullet in bullets:
        movement = bullet.move(player)
        if movement == "out":
            rem.append(bullet)
            score += 1
        if movement == "crash":
            hearts.heart -= 1
            bullets.remove(bullet)
            if hearts.heart == 0:
                pygame.quit()
                # win.quit()
                with open("scores.txt", "a") as f:
                    f.write(username + ": score = " + str(score) + ", successful shots = " + str(player.shots) + "\n")
                end_menu(score)
                return
    for shoot in shoots:
        movement = shoot.move(player, bullets)
        if movement == "crash":
            score += 10
            player.shots += 1
    for bullet in rem:
        try:
            bullets.remove(bullet)
        except ValueError:
            pass

    while len(bullets) != round(n):
        bullets = adjust_bullets(bullets, n)

    return hearts, player, shoots, bullets, score, n


def move_objs(hearts, player, shoots, bullets, score, n, clouds):
    player.move()
    clouds.move()
    return move_bullets(hearts, player, shoots, bullets, score, n)


def main(win, username):
    clock = pygame.time.Clock()
    hearts = Heart()
    player = Player(random.randint(5, 35), random.randint(0, HEIGHT - 50), 100, 50)
    clouds = Clouds()
    shoots = player.shoots
    n = 5
    bullets = create_bullets(n)
    score = 0
    try:
        with open("max_score.txt", "r") as f:
            line = f.readline()
            max_score = int(line)
    except FileNotFoundError:
        max_score = 0
    run = True
    playing = False
    while run:
        clock.tick(FPS)
        draw_window(win, hearts, player, shoots, clouds, bullets, score, max_score, playing)
        n = min(30, n)
        if playing:
            n += 0.001
            try:
                hearts, player, shoots, bullets, score, n = move_objs(hearts, player, shoots, bullets, score, n, clouds)
            except:
                break
        if max_score < score:
            max_score = score
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                with open("max_score.txt", "w") as f:
                    f.write(str(max_score))
                run = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    playing = True if playing == False else False
        pygame.display.update()
    return


#Create an instance of Tkinter frame
win = Tk()
win.title("Welcome! ^^")

#Set the geometry of Tkinter frame
win.geometry("550x200")

clicked = False
username = ""

def close():
   global clicked, username
   clicked = True
   username = entry.get()
   win.destroy()

#Initialize a Label to display the User Input
label=Label(win, text="What's your name? :)", font=("Courier 18 bold"))
label.pack(pady=20)

# Add an empty label to create vertical spacing
empty_label = Label(win, height=1)
empty_label.pack()

#Create an Entry widget to accept User Input
entry = Entry(win, width=40)
entry.focus_set()
entry.pack()

#Create a Button to validate Entry Widget
ttk.Button(win, text="Okay", width= 20, command=close).pack(pady=30)

win.mainloop()

WIDTH, HEIGHT = 1000, 700
FPS = 60

if clicked:
    pygame.init()

    screen_width = tk.Tk().winfo_screenwidth()
    screen_height = tk.Tk().winfo_screenheight()

    os.environ["SDL_VIDEO_WINDOW_POS"] = "%d,%d" % (
        screen_width // 2 - WIDTH // 2,
        screen_height // 2 - HEIGHT // 2,
    )
    WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Dodger Plane")
    pygame.display.set_icon(pygame.image.load(r"E:\classes\multimedia\final project\game part\assets\icon.png"))

    PLAYER_BACKFORTH = True

    FONT = pygame.font.SysFont("comicsans", 30)
    BIG_FONT = pygame.font.SysFont("comicsans", 70)


    BLACK = (0, 0, 0)
    main(WINDOW, username)



Training Accuracy: 1.0
Validation Accuracy: 0.9444444444444444
Moving the player down. Player position: [0, -1]
Training Accuracy: 1.0
Validation Accuracy: 0.9444444444444444
Moving the player right. Player position: [1, 0]
Training Accuracy: 1.0
Validation Accuracy: 0.9444444444444444
Moving the player down. Player position: [0, -1]
Training Accuracy: 1.0
Validation Accuracy: 0.9444444444444444
Moving the player left. Player position: [-1, 0]
Training Accuracy: 1.0
Validation Accuracy: 0.9444444444444444
Moving the player down. Player position: [0, -1]
Training Accuracy: 1.0
Validation Accuracy: 0.9444444444444444
Moving the player down. Player position: [0, -1]
Training Accuracy: 1.0
Validation Accuracy: 0.9444444444444444
Moving the player down. Player position: [0, -1]
Training Accuracy: 1.0
Validation Accuracy: 0.9444444444444444
Moving the player left. Player position: [-1, 0]
